In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, LeaveOneGroupOut, ParameterGrid
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
ppg_dataframes = {}
for i in np.arange(1, 16):
    name = f"data{i}"
    acc = pd.read_csv(f"/content/drive/My Drive/ResearchProject/HAR/HAR/ppg+dalia/data/PPG_FieldStudy/S{i}/S{i}_E4/ACC_specific_windowed_features.csv")
    hr = pd.read_csv(f"/content/drive/My Drive/ResearchProject/HAR/HAR/ppg+dalia/data/PPG_FieldStudy/S{i}/S{i}_E4/HR_specific_windowed_features.csv")
    bvp = pd.read_csv(f"/content/drive/My Drive/ResearchProject/HAR/HAR/ppg+dalia/data/PPG_FieldStudy/S{i}/S{i}_E4/BVP_specific_windowed_features.csv")
    acc['start_time'] = pd.to_datetime(acc['start_time'])
    hr['start_time'] = pd.to_datetime(hr['start_time'])
    bvp['start_time'] = pd.to_datetime(bvp['start_time'])
    acc['end_time'] = pd.to_datetime(acc['end_time'])
    hr['end_time'] = pd.to_datetime(hr['end_time'])
    bvp['end_time'] = pd.to_datetime(bvp['end_time'])

    common_columns = ['start_time', 'end_time', 'activity']
    data = acc.merge(hr, on=common_columns, how='inner').merge(bvp, on=common_columns, how='inner')
    allFeatures  = [col for col in data.columns if col not in ["start_time", "end_time"]]
    data = data[allFeatures]
    data = data[~data['activity'].isin([0])]
    data.loc[data['activity'].isin([2, 7, 8, 9]), 'activity'] = 0
    data.loc[data['activity'].isin([3, 4, 5, 6]), 'activity'] = 1
    ppg_dataframes[name] = data

bigideas_dataframes = {}
for i in np.arange(1, 17):
    if i < 10:
        name = f"data{i}"
        acc = pd.read_csv(f"/content/drive/My Drive/ResearchProject/BigIdeas/00{i} (1)/ACC_simple_windowed.csv")
        hr = pd.read_csv(f"/content/drive/My Drive/ResearchProject/BigIdeas/00{i} (1)/HR_simple_windowed.csv")
        bvp = pd.read_csv(f"/content/drive/My Drive/ResearchProject/BigIdeas/00{i} (1)/BVP_simple_windowed.csv")
    if i > 9:
        name = f"data{i}"
        acc = pd.read_csv(f"/content/drive/My Drive/ResearchProject/BigIdeas/0{i} (1)/ACC_simple_windowed.csv")
        hr = pd.read_csv(f"/content/drive/My Drive/ResearchProject/BigIdeas/0{i} (1)/HR_simple_windowed.csv")
        bvp = pd.read_csv(f"/content/drive/My Drive/ResearchProject/BigIdeas/0{i} (1)/BVP_simple_windowed.csv")
    acc['start_time'] = pd.to_datetime(acc['start_time'])
    hr['start_time'] = pd.to_datetime(hr['start_time'])
    bvp['start_time'] = pd.to_datetime(bvp['start_time'])
    acc['end_time'] = pd.to_datetime(acc['end_time'])
    hr['end_time'] = pd.to_datetime(hr['end_time'])
    bvp['end_time'] = pd.to_datetime(bvp['end_time'])

    common_columns = ['start_time', 'end_time']
    data = acc.merge(hr, on=common_columns, how='inner').merge(bvp, on=common_columns, how='inner')
    allFeatures  = [col for col in data.columns if col not in ["start_time", "end_time"]]
    data = data[allFeatures]
    data.columns = data.columns.str.replace(' ', '', regex=False)
    bigideas_dataframes[name] = data

# Create indices for splitting back the combined data
ppg_indices = list(range(len(pd.concat(ppg_dataframes.values(), ignore_index=True))))
bigideas_indices = list(range(len(pd.concat(bigideas_dataframes.values(), ignore_index=True))))

# Combine all data into a single dataframe for UMAP
combined_df = pd.concat([pd.concat(ppg_dataframes.values(), ignore_index=True),
                         pd.concat(bigideas_dataframes.values(), ignore_index=True)], ignore_index=True)
numerical_cols = [col for col in combined_df.columns if col != 'activity']
scaled_data = StandardScaler().fit_transform(combined_df[numerical_cols])
print(scaled_data.columns)
"""# Compute UMAP
umap_reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=3)
X_umap = umap_reducer.fit_transform(scaled_data)
umap_df = pd.DataFrame(data=X_umap, columns=[f'UMAP{i}' for i in np.arange(1, 4)])

# Separate the data back into ppg_dataframes and bigideas_dataframes
ppg_df_umap = umap_df.iloc[ppg_indices].reset_index(drop=True)
bigideas_df_umap = umap_df.iloc[bigideas_indices].reset_index(drop=True)

# Add the activity column back to ppg_df_umap
ppg_df_umap['activity'] = combined_df.iloc[ppg_indices]['activity'].reset_index(drop=True)

# Update the original dictionaries with the UMAP data
start_idx = 0
for key, df in ppg_dataframes.items():
    end_idx = start_idx + len(df)
    ppg_dataframes[key] = ppg_df_umap.iloc[start_idx:end_idx].reset_index(drop=True)
    start_idx = end_idx

start_idx = 0
for key, df in bigideas_dataframes.items():
    end_idx = start_idx + len(df)
    bigideas_dataframes[key] = bigideas_df_umap.iloc[start_idx:end_idx].reset_index(drop=True)
    start_idx = end_idx"""

KeyError: 'start_time'